# Vitality Project

**Project Name:** Vitality Health Data Analysis  
**Author(s):** [Your Name / Team Name]  
**Date:** 2026-01-06  
**Course / Organization:** ITS 2122 / [Organization Name]  
**Notebook Purpose:**  
This notebook contains the full data analysis and cleaning pipeline for the Vitality Project.  
It includes:
- Data loading and inspection  
- Data cleaning and preprocessing  
- Exploratory Data Analysis (EDA)  
- Feature engineering for predictive modeling  
- Merging and mapping categorical IDs  
- Final cleaned dataset export  

**Data Sources:**  
- Raw data: `diabetic_data.csv`  
- Mapping IDs: `IDs_mapping.csv`  

**Notes:**  
- Missing values are handled appropriately.  
- Expired patients are removed from the dataset.  
- Weight column is dropped due to high missing values.  
- ID mappings for admission, discharge, and source are merged.  

**References / Tools:**  
- Python 3  
- Pandas, NumPy  
- Jupyter Notebook  
